In [34]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import numpy as np
from agents.knn import *
from settings import settings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
data = parse_dataset('/home/kku/Dropbox/CG_track_3')
state_keys = settings['state_keys']
states, actions = extract_parameters(data)
Y, actions = unify_actions(actions)
X = np.array(states)
Y = np.array(Y)

In [45]:
np.unique(Y)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [42]:
state_keys = settings['state_keys']
states, actions = extract_parameters(data)
# self.actions = actions
# Reducinf actions set
Y, actions = unify_actions(actions)

X = np.array(states)
X, Y = sklearn.utils.shuffle(X, np.array(Y))


In [46]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.20)

In [ ]:
%%time
scaler = StandardScaler()  
scaler.fit(X_train)  
classifier = MLPClassifier(hidden_layer_sizes=(800,200), )
classifier.fit(scaler.transform(X_train), Y_train) 

In [41]:
classifier.score(scaler.transform(X_test), Y_test)

0.4965786901270772

In [ ]:
def dump():
    path = 'saved_model'
    print(f'Dumping model to: {path}')
    np.savez(
        'saved_model/parameters.npz',
        **{
            'actions': actions,
            'state_keys': state_keys
        }
    
    )
    joblib.dump(classifier, 'saved_model/classifier')
    joblib.dump(scaler, 'saved_model/scaler')
    
dump()
    